# Graph Theory in Python: Routing example

## Try me
[![Open In Colab](../../_static/colabs_badge.png)](https://colab.research.google.com/github/ffraile/operations-research-notebooks/blob/main/docs/source/MIP/tutorials/Routing%20example.ipynb)[![Binder](../../_static/binder_badge.png)](https://mybinder.org/v2/gh/ffraile/operations-research-notebooks/main?labpath=docs%2Fsource%2FMIP%2Ftutorials%2FRouting%20example.ipynb)

## Description
In this notebook, you will learn how to solve network problems using Python. For the set-up, we will use a map of the city of Valencia, and calculate the shortest distance between two points using the actual street map of the city.
We will use the following libraries: 

- [Networkx](https://networkx.org/): This Python library implements a lot of network theory utils and algorithms and is the main library used in the problems. 

- [Osmnx](https://osmnx.readthedocs.io/en/stable/): This library allows us to create a graph from an Open Street Maps query (similar to what you would use to find a place in Google Maps).

Additionally, we will use the following libraries to render the maps:

- [Folium](https://python-visualization.github.io/folium/index.html): This library allows us to render the maps in our Notebooks

Remember you must **Trust** the notebook to test it.
In Colabs, Run the following cell to install the libraries:

In [ ]:
!pip install networkx[default,extra]
!apt-get -qq install -y libspatialindex-dev
!pip install osmnx
!pip install folium
!pip install python-igraph
!pip install mapclassify

## Network creation
First we need to create the map:

In [ ]:
import networkx as nx
import osmnx as ox
from IPython.display import IFrame
%matplotlib inline


# Get the map of Valencia
G_nx = ox.graph_from_place('Valencia, VC, Spain', network_type='drive')

print("The streets of Valencia can be modeled with a graph of size: " + str(G_nx.size()))

### Code Explanation
We have created a graph that represents the streets of Valencia, Spain, using the function `ox.graph_from_place`. The function takes as input a place name, and returns a graph of the streets in that place. The parameter `network_type` specifies the type of streets that we want to include in the graph. In this case, we are only interested in streets that can be used by cars. Other possible values are `walk` (pedestrian streets), `bike` (streets that can be used by bikes), `drive_service` (service roads), `all` (all the streets in the place).

Now, the object G_nx is a graph of the city, we do some adaptations that will allow us to work with the map more easily:

In [ ]:
# convert osmids IDs to a list
osmids = list(G_nx.nodes)

# Relabel the nodes with integers. This facilitates handling the tree
G_nx = nx.relabel.convert_node_labels_to_integers(G_nx)

# give each node its original osmid as attribute, since we relabeled them
osmid_values = {k:v for k, v in zip(G_nx.nodes, osmids)}
nx.set_node_attributes(G_nx, osmid_values, 'osmid')



## Rendering the map
Let us now plot the map in an interactive component:


In [ ]:
ox.graph_to_gdfs(G_nx, nodes=False).explore()


### Code explanation
The function `ox.graph_to_gdfs` converts the graph to a GeoDataFrame, which is a data structure that allows us to plot the graph in an interactive map. The parameter `nodes=False` indicates that we are not interested in plotting the nodes of the graph (only the edges).
Any GeoDataframe can be plotted using the function `explore()`. As we will see in the next section, we can also explore routes in the map using this function.

## Shortest path
Let us now define a function that uses this map to calculate the shortest path. 


In [ ]:
def get_route(source, destination):
    """
    Given two coordinates (source and destination) calculate the shortest path between the closest nodes to the source and
    the closest node to the destination.
     A point is a tuple of two double values specifying latitude and longitude (e.g. point_1 = (39.464060144309364,
     -0.3624288516715025)).

    Args:
        source - A tuple containing latitude and longitude values of the source point.
        destination - A tuple containing latitude and longitude values of the source point.

    Returns: 
        route: The shortest path along the map
        distance: the minimum distance in meters (summation of great-circle distance between nodes in the shortest path)
    """
    # Get the nearest node to source
    node_1 = ox.distance.nearest_nodes(G_nx, X=source[0], Y=source[1])

    # Get the nearest node to destination
    node_2 = ox.distance.nearest_nodes(G_nx, X=destination[0], Y=destination[1])

    # Get the shortest path between source and destination nodes
    route = ox.shortest_path(G_nx, node_1, node_2)

    # Return the route and the nearest nodes

    return route, [node_1, node_2]

### Code Explanation
The function above uses the following functions from the libraries:

- `ox.distance.nearest_nodes(G_nx, X=source[0], Y=source[1])`: This function returns the nearest node to a given point. The point is specified by its latitude and longitude.
- `ox.shortest_path(G_nx, node_1, node_2)`: This function returns the shortest path between two nodes in the graph.


The function basically uses the previous functions to first find the nearest nodes to the source and destination points, and then calculates the shortest path between them. Finally, it returns the shortest path and the nearest nodes of the graph which are closer to the provided coordinates.


### Test between two points

 Let us test the function!. From Google Maps, it is very easy to get the coordinates of a point. Just right-click on the map, and it will show the coordinates. After, you can hover over the coordinates and left click to copy the coordinates to your clipboard. Now, if you paste in Google Colabs, you will get the coordinates of the point.

![ Coordinates](./img/getting_coordinates_from_google_maps.png){width=50%}

Note that the coordinates are in the form (latitude, longitude). However, the function `get_route` expects the coordinates in the form (longitude, latitude). Therefore, we need to invert the coordinates before passing them to the function.

Here is an example of two points in Valencia:

- [EDEM](https://edem.eu/): Coordinates (-0.3288013882525529, 39.46211739713285)
- [CIGIP Research Center @ UPV](https://cigip.webs.upv.es/index.php/en/): Coordinates (-0.3344765250611888, 39.46211739713285)

In [ ]:
edem = (-0.3288013882525529, 39.46211739713285)
cigip = (-0.3346191716152525, 39.47696577364303)

route, nodes = get_route(edem, cigip)


### Converting to GeoDataFrame
Now, we can convert the route to a GeoDataFrame, which is a dataframe containing the information of the route. This will allow us for instance to calculate the total distance or to plot the route in a map.
The function used to convert the route to a GeoDataFrame is `ox.routing.route_to_gdf`. The function takes as input the graph, the route and the attribute that we want to use to calculate the distance. In this case, we are using the attribute `length`, which is the length of the street in meters.

In [ ]:
route_edges = ox.routing.route_to_gdf(G_nx, route, "length")
display(route_edges.head())

### Calculating the distance
Now, we can calculate the distance of the route by summing the length of the streets in the route. The length of the streets is stored in the column `length` of the GeoDataFrame. Therefore, we can use the function `sum` to calculate the total distance.

In [ ]:
total_distance = sum(route_edges["length"])
print("The total distance between the two points is: " + str(total_distance) + " meters")

### Exploring the route
Finally, we can plot the route in a map using the function `explore` of the GeoDataFrame. The function takes as input the map where we want to plot the route. We can use the argument `color` to specify the color of the route, the argument `style_kwds` to define style properties, and the argument `m` to specify the map where we want to plot the route. If we do not specify the map, the function will create a new map. In the example below we set the weight of the route to 5 to plot it in a thicker line.

In [ ]:
route_edges.explore( style_kwds={"weight": 5})

## Getting the routes of a distribution network
We can use the function `get_route`and the information contained in Geo Dataframes to get the routes of a distribution network. For instance, let us calculate the routes for a distribution network with two sources and two destinations in Valencia.

In [ ]:
sources = [
    {
        "id": 1,
        "coordinates": (-0.3783881019008758, 39.47875115874353)},
    {
        "id": 2,
        "coordinates": (-0.3929897392129934, 39.4817119703732)
    }]

destinations = [
    {
        "id": 1,
        "coordinates": (-0.3346191716152525, 39.47696577364303)},
    {
        "id": 2,
        "coordinates": (-0.3288013882525529, 39.46211739713285)
    }]

# Init the routes and distances
distances = []

routes = []

# Iterate over the sources and destinations
for source in sources:
    for destination in destinations:
        route, nodes = get_route(source["coordinates"], destination["coordinates"])
        route_gdf = ox.routing.route_to_gdf(G_nx, route, "length")
        routes.append(
            {
                "source": source["id"],
                "destination": destination["id"],
                "route": route_gdf
            }
        )
        distance = sum(route_gdf["length"])
        distances.append(
            {
                "source": source["id"],
                "destination": destination["id"],
                "distance": distance
            }
        )


### Code Explanation
In the code above, we have defined two dictionaries with the information of sources and destinations, and then we have iterated over them to calculate the routes and distances, using the `get_route` function defined before. We have stored the different routes in a list of GeoDataFrames, and the distances in a list of dictionaries. The dictionaries contain the information of the source, destination and distance of each route.

Now, we can for instance display a dataframe with the distances:


In [ ]:
import pandas as pd
pd.DataFrame(distances)


We can also plot the routes using the following script:

In [ ]:
# Define a color map so that the routes have different colors depending on the source
# we use source ids as keys and colors as values. The colors are semi-transparent so that we can see the routes even if they overlap
color_map = {
    1: "#ff000055",
    2: "#0000ff55"
}

# Add the first route to a map
m = routes[0]["route"].explore(color=color_map[routes[0]["source"]],  style_kwds={"weight": 5})

# Add the rest of the routes to the map
for route in routes[1:]:
    route["route"].explore(color=color_map[route["source"]],  style_kwds={"weight": 5}, m=m)

# Display the map
m

## Analysis questions
1. Try the scripts with other points in the map. What is the shortest path between them?
2. What are the main limitations of this approach? How could we improve it?
3. Try to get the distance using a mapping application (e.g. Google Maps). What is the difference between the distance calculated by the script and the distance calculated by the application? Why?
4. Ask an AI assistant to describe the concept of Manhattan distance to you. Calculate the Manhattan distance between the two points. What is the difference between the distance calculated by the script and the distance calculated by the application? Why? Could you use the Manhattan distance to improve the script?
